# 침수 API용 데이터 구축

In [1]:
import pandas as pd 
import glob
from tqdm import tqdm
import dask.dataframe as dd

def make_sido_code(row):
    return str(row["시군구코드"])[:2]

### (1) 건축물대장 기본개요

In [64]:
%%time

# 건축물대장 기본개요
columns = ["관리건축물대장PK","관리상위건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명", \
            "대지위치","도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명","블록","로트","외필지수", \
            "새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번","새주소부번","지역코드","지구코드","구역코드", \
            "지역코드명","지구코드명","구역코드명","생성일자"]

basis = dd.read_csv("data/rawdata/building/기본개요/mart_djy_01.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
print(f"건축물대장 기본개요의 분할된 개수는 {basis.npartitions}개입니다.")

busan_basis = basis[["관리건축물대장PK", "관리상위건축물대장PK", "시군구코드"]]
busan_basis["시도코드"] = busan_basis.apply(make_sido_code, axis=1, meta=object)
busan_basis = busan_basis[busan_basis["시도코드"] == "26"].drop(["시군구코드"], axis=1)

# preprocessing
busan_basis["관리건축물대장PK"] = busan_basis["관리건축물대장PK"].map(lambda x: x.strip())
busan_basis["관리상위건축물대장PK"] = busan_basis["관리상위건축물대장PK"].map(lambda x: x.strip())
busan_basis = busan_basis.drop("시도코드", axis=1)

# convert dask to pandas
busan_basis = busan_basis.compute()

# save csv
busan_basis.to_csv("data/refined-data/busan-기본개요.csv", encoding="utf-8", index=False)

건축물대장 기본개요의 분할된 개수는 80개입니다.
Wall time: 9min


### (2) 건축물대장 주택가격

In [65]:
%%time

# 건축물대장 주택가격

# names=columns
# columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치","도로명대지위치", \
#             "건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명","블록","로트","외필지수","새주소도로코드", \
#             "새주소지상지하코드","새주소본번","새주소부번","기준일자","주택가격","생성일자"]

price = dd.read_csv("data/rawdata/building/주택가격/mart_djy_08.txt", sep="\|", dtype=str, encoding='cp949',
                        on_bad_lines="skip", header=None, engine='python')

# the number of partitions
print(f"건축물대장의 주택가격이 분할된 파일 개수는 {price.npartitions}개입니다.")

# save partitions
# price.to_csv("data/rawdata/building/주택가격/csvs", encoding="utf-8")

price_df = price[[0,8,23,24]]
price_df.columns = ["관리상위건축물대장PK","시군구코드","주택가격","생성일자"]
price_df["시도코드"] = price_df.apply(make_sido_code, axis=1, meta=object)
price_df = price_df.loc[price_df["시도코드"] == "26"]

# preprocessing
price_df["관리상위건축물대장PK"] = price_df["관리상위건축물대장PK"].map(lambda x: x.strip())
price_df = price_df.map_partitions(lambda df: df.sort_values(by="생성일자", ascending=False))
price_df = price_df.drop(["시도코드"], axis=1).drop_duplicates()

# convert dask to pandas
busan_price_pdf = price_df.compute()

# save csv
busan_price_pdf.to_csv("data/refined-data/busan-price.csv", encoding="utf-8", index=False)

# row 개수
print(f"주택가격의  행수는 {busan_price_pdf.shape[0]}개입니다.")

건축물대장의 주택가격이 분할된 파일 개수는 416개입니다.
주택가격의  행수는 9183677개입니다.
Wall time: 43min 27s


### (3) 건축물대장 표제부

In [102]:
%%time

columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치", \
            "도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명", \
            "블록","로트","외필지수","새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번", \
            "새주소부번","동명","주부속구분코드","주부속구분코드명","대지면적(M2)","건축면적(M2)", \
            "건폐율(%)","연면적","용적률산정연면적","용적률","구조코드","구조코드명","기타구조", \
            "주용도코드","주용도코드명","기타용도","지붕코드","지붕코드명","기타지붕","세대수(세대)", \
            "가구수(가구)","높이(M)","지상층수","지하층수","승용승강기수","비상용승강기수","부속건축물수", \
            "부속건축물면적","총동연면적","옥내기계식대수(대)","옥내기계식면적","옥외기계식대수(대)", \
            "옥외기계식면적","옥내자주식대수(대)","옥내자주식면적","옥외자주식대수(대)","옥외자주식면적", \
            "허가일","착공일","사용승인일","허가번호년","허가번호기관코드","허가번호기관코드명", \
            "허가번호구분코드","허가번호구분코드명","호수(호)","에너지효율등급","에너지절감율", \
            "에너지EPI점수","친환경건축물등급","친환경건축물인증점수","지능형건축물등급","지능형건축물인증점수", \
            "생성일자","내진설계적용여부","내진능력"]

building = dd.read_csv("data/rawdata/building/표제부/mart_djy_03.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
print(f"건축물대장 표제부의 분할된 개수는 {building.npartitions}개입니다.")

busan_building = building[["관리건축물대장PK","대장종류코드명","도로명대지위치","시군구코드","시군구코드", \
                            "대지면적(M2)","건축면적(M2)","건폐율(%)", "연면적","주용도코드명","가구수(가구)","높이(M)","지상층수","지하층수", \
                            "사용승인일","허가번호기관코드","친환경건축물등급","친환경건축물인증점수","지능형건축물등급","지능형건축물인증점수","내진설계적용여부","내진능력"]]

busan_building["시도코드"] = building.apply(make_sido_code, axis=1, meta=object)
busan_building = busan_building[busan_building["시도코드"] == "26"].drop(["시군구코드"], axis=1)

# preprocessing
busan_building["관리건축물대장PK"] = busan_building["관리건축물대장PK"].map(lambda x: x.strip())
busan_building = busan_building.drop("시도코드", axis=1)

# convert dask to pandas
busan_building = busan_building.compute()

# save csv
busan_building.to_csv("data/refined-data/busan-표제부.csv", encoding="utf-8", index=False)

건축물대장 표제부의 분할된 개수는 47개입니다.
Wall time: 5min 32s


### (4) KISTI가 제공하는 침수된 건물 정보

In [2]:
# KISTI 침수된 건물 정보
path = "data/rawdata/flooding-kisti-data/*.csv"

# Merge all files
flood_df = pd.concat([pd.read_csv(f, encoding="utf-8", dtype=str, keep_default_na=False) for f in glob.glob(path)])
flood_df = flood_df.drop(["OBJECTID_1", "OBJECTID", "full_pk", "pnu_1", "Shape_Length", "Shape_Area"], axis=1)
flood_df["관리건축물대장PK"] = flood_df["관리건축물대장PK"].apply(lambda x: x.strip())
print(flood_df.shape)

# Convert pandas to dask dataframe
flood_dd = dd.from_pandas(flood_df, npartitions=3)
flood_dd.head()

(141129, 15)


,PNU,BD_MGT_SN,시군구코드,법정동코드,번,지,관리건축물대장PK,건물명,새주소도로코드,새주소법정동코드,새주소지상지하코드,동명칭,대지구분코드,Cent_X,Cent_Y
0,2653010700102740020,,26530,10700,0274,0020,26530-22997,,265304217371,10701,0,,1,381261,185051
0,2611012000100140072,2611012000100140072008066,26110,12000,0014,0072,26110-7752,,261104175158,12001,0,,1,384394,180636
0,2611012000100140072,2611012000100140072008066,26110,12000,0014,0072,26110-7752,,261104175158,12001,0,,1,384394,180636
0,2611012000100140072,2611012000100140072008066,26110,12000,0014,0072,26110-7752,,261104175158,12001,0,,1,384394,180636
0,2614012400105830003,2614012400105830003018981,26140,12400,0583,0003,26140-24148,,,,0,,1,383781,176883


### 모든 데이터 병합

In [3]:
# 건축물대장 기본개요
busan_basis = dd.read_csv("data/refined-data/busan-기본개요.csv", encoding="utf-8", dtype=str, keep_default_na=False)

# 첫번째 병합
merged =  flood_dd.merge(busan_basis, how="left", on=["관리건축물대장PK"])

# 건축물대장 주택가격
busan_price = dd.read_csv("data/refined-data/busan-price.csv", encoding="utf-8", dtype=str, keep_default_na=False)
busan_price = busan_price.drop(["시군구코드"], axis=1)

# 두번째 병합
merged_price = merged.merge(busan_price, how="left", on=["관리상위건축물대장PK"])

#건축물대장 표제부
busan_building = dd.read_csv("data/refined-data/busan-표제부.csv", encoding="utf-8", dtype=str, keep_default_na=False)

# 마지막 병합
final_df = merged_price.merge(busan_building, how="left", on=["관리건축물대장PK"])
final_df = final_df.compute()
final_df = final_df.replace({"None": None, "": None})

final_df.to_csv("final-kisti-flooding-data.csv", encoding="utf-8", index=False)

In [5]:
df = pd.read_csv("final-kisti-flooding-data.csv", encoding="utf-8", dtype=str, keep_default_na=False)
print(df.columns)
df.head()

Index(['PNU', 'BD_MGT_SN', '시군구코드', '법정동코드', '번', '지', '관리건축물대장PK', '건물명',
       '새주소도로코드', '새주소법정동코드', '새주소지상지하코드', '동명칭', '대지구분코드', 'Cent_X', 'Cent_Y',
       '관리상위건축물대장PK', '주택가격', '생성일자', '대장종류코드명', '도로명대지위치', '대지면적(M2)',
       '건축면적(M2)', '건폐율(%)', '연면적', '주용도코드명', '가구수(가구)', '높이(M)', '지상층수',
       '지하층수', '사용승인일', '허가번호기관코드', '친환경건축물등급', '친환경건축물인증점수', '지능형건축물등급',
       '지능형건축물인증점수', '내진설계적용여부', '내진능력'],
      dtype='object')


,PNU,BD_MGT_SN,시군구코드,법정동코드,번,지,관리건축물대장PK,건물명,새주소도로코드,새주소법정동코드,...,지상층수,지하층수,사용승인일,허가번호기관코드,친환경건축물등급,친환경건축물인증점수,지능형건축물등급,지능형건축물인증점수,내진설계적용여부,내진능력
0,2632010500108570003,2632010500108570003004348,26320,10500,0857,0003,26320-16550,벽산그린필드,263204196206,10501,...,15,1,19920727,,,0,,0,1,
1,2623010400108900001,2623010400108900001022921,26230,10400,0890,0001,26230-23256,부산진초등학교 본관동,262303129019,10401,...,5,1,20060320,,,0,,0,1,
2,2629010600108670002,,26290,10600,,,26290-6398,,262904193304,10601,...,1,0,19851129,,,0,,0,0,
3,2629010600108670002,2629010600108670002001921,26290,10600,0867,0002,26290-6399,,262904193304,10601,...,2,0,19851129,,,0,,0,0,
4,2614012000100010038,2614012000100010038022816,26140,12000,0001,0038,26140-17908,,,,...,2,0,19850214,,,0,,0,0,
